## A simple set of queries to fetch data from IBM Db2 database

In [33]:
import ibm_db
import pandas as pd 
import ibm_db_dbi
import ibm_db_sa
import sqlalchemy

Data Source:
Snapshots for the three datasets in .CSV format can be downloaded from the following links:

Chicago Socioeconomic Indicators: https://ibm.box.com/shared/static/05c3415cbfbtfnr2fx4atenb2sd361ze.csv
Chicago Public Schools: https://ibm.box.com/shared/static/f9gjvj1gjmxxzycdhplzt01qtz0s7ew7.csv
Chicago Crime Data: https://ibm.box.com/shared/static/svflyugsr9zbqy5bmowgswqemfpm1x7f.csv
        
Original Data:
1. Chicago Socioeconomic Indicators

This [dataset](https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2) contains a selection of six socioeconomic indicators of public health significance and a “hardship index,” by Chicago community area, for the years 2008 – 2012.

2. Chicago Public Schools

This [dataset](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t) shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year.

3. Chicago Crime Data

This [dataset](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2) reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days.



In [2]:
%load_ext sql

In [34]:
# this does not work if jupyter is already connected to IBM db2 using API.
%sql ibm_db_sa://gsv42017:wlmx0qnkw13xds%5E2@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
            

'Connected: gsv42017@BLUDB'

In [4]:
pd.set_option('display.max_columns',0)


In [27]:
#Following 3-4 cells are to connect to ibm-db2 using API and a wrapper to use with pandas

In [366]:
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = "dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net" # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_uid = "gsv42017"        # e.g. "abc12345"
dsn_pwd = "wlmx0qnkw13xds^2"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "50000"                # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"

In [367]:
#DO NOT MODIFY THIS CELL. 
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

#print the connection string to check correct values are specified
print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=gsv42017;PWD=wlmx0qnkw13xds^2;


In [368]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create database connection

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )


Connected to database:  BLUDB as user:  gsv42017 on host:  dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net


In [369]:
# to read the sql query results into a dataframe, the connection 
# needs to be wrapped in an ibm_db_dbi object

pconn = ibm_db_dbi.Connection(conn)


In [ ]:
# reading all tables into pandas dataframes
crime = "select * from chicago_crime_data"
df_crime = pd.read_sql(crime,pconn)


census = "select * from census_data"
df_census = pd.read_sql(census,pconn)

public_schools = "select * from chicago_public_schools"
df_school = pd.read_sql(public_schools,pconn)

### Q1. Find the total number of crimes recorded in the crime table.

In [7]:
#Q1
%%sql
select count(Case_Number) from CHICAGO_CRIME_DATA

 * ibm_db_sa://gsv42017:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


1
533


In [257]:
# construct query
sqlQ1 = "select count(Case_Number) from CHICAGO_CRIME_DATA"


In [36]:
#execute the query
Q1stmt = ibm_db.exec_immediate(conn, sqlQ1)

In [38]:
# make sure pconn (ibm_db_dbi) is used instead of the original connection created using ibm_db
df_Q1 = pd.read_sql(sqlQ1, pconn)

In [39]:
# print results of the query
df_Q1

,1
0,533.0


### Q2. Retrieve first 10 rows from the CRIME table.

In [31]:
# Q2
%sql select * from CHICAGO_CRIME_DATA LIMIT 10;

# Alternate query
%sql select * from CHICAGO_CRIME_DATA FETCH FIRST 10 ROWS ONLY;

 * ibm_db_sa://gsv42017:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


id,case_number,DATE,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area_number,fbicode,x_coordinate,y_coordinate,YEAR,updatedon,latitude,longitude,location
3512276,HK587712,08/28/2004 05:50:56 PM,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,FALSE,FALSE,911,9,14,58,6,1155838,1873050,2004,02/10/2018 03:50:01 PM,41.80744050,-87.70395585,"(41.8074405, -87.703955849)"
3406613,HK456306,06/26/2004 12:40:00 PM,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,FALSE,FALSE,1112,11,27,23,6,1152206,1906127,2004,02/28/2018 03:56:25 PM,41.89827996,-87.71640551,"(41.898279962, -87.716405505)"
8002131,HT233595,04/04/2011 05:45:00 AM,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,FALSE,FALSE,221,2,3,38,6,1177436,1876313,2011,02/10/2018 03:50:01 PM,41.81593313,-87.62464213,"(41.815933131, -87.624642127)"
7903289,HT133522,12/30/2010 04:30:00 PM,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,FALSE,FALSE,423,4,7,46,6,1194622,1850125,2010,02/10/2018 03:50:01 PM,41.74366532,-87.56246276,"(41.743665322, -87.562462756)"
10402076,HZ138551,02/02/2016 07:30:00 PM,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,FALSE,FALSE,831,8,15,66,6,1155240,1860661,2016,02/10/2018 03:50:01 PM,41.77345530,-87.70648047,"(41.773455295, -87.706480471)"
7732712,HS540106,09/29/2010 07:59:00 AM,006XX W CHICAGO AVE,810,THEFT,OVER $500,PARKING LOT/GARAGE(NON.RESID.),FALSE,FALSE,1323,12,27,24,6,1171668,1905607,2010,02/10/2018 03:50:01 PM,41.89644677,-87.64493868,"(41.896446772, -87.644938678)"
10769475,HZ534771,11/30/2016 01:15:00 AM,050XX N KEDZIE AVE,810,THEFT,OVER $500,STREET,FALSE,FALSE,1713,17,33,14,6,1154133,1933314,2016,02/10/2018 03:50:01 PM,41.97284491,-87.70860008,"(41.972844913, -87.708600079)"
4494340,HL793243,12/16/2005 04:45:00 PM,005XX E PERSHING RD,860,THEFT,RETAIL THEFT,GROCERY FOOD STORE,TRUE,FALSE,213,2,3,38,6,1180448,1879234,2005,02/28/2018 03:56:25 PM,41.82387989,-87.61350386,"(41.823879885, -87.613503857)"
3778925,HL149610,01/28/2005 05:00:00 PM,100XX S WASHTENAW AVE,810,THEFT,OVER $500,STREET,FALSE,FALSE,2211,22,19,72,6,1160129,1838040,2005,02/28/2018 03:56:25 PM,41.71128051,-87.68917910,"(41.711280513, -87.689179097)"
3324217,HK361551,05/13/2004 02:15:00 PM,033XX W BELMONT AVE,820,THEFT,$500 AND UNDER,SMALL RETAIL STORE,FALSE,FALSE,1733,17,35,21,6,1153590,1921084,2004,02/28/2018 03:56:25 PM,41.93929582,-87.71092344,"(41.939295821, -87.710923442)"


In [42]:
sqlQ2 = "select * from CHICAGO_CRIME_DATA LIMIT 10"

In [43]:

df_Q2 = pd.read_sql_query(sqlQ2,pconn)

In [44]:
df_Q2

,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,...,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,UPDATEDON,LATITUDE,LONGITUDE,LOCATION
0,3512276,HK587712,08/28/2004 05:50:56 PM,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,FALSE,FALSE,...,14,58,6,1155838,1873050,2004,02/10/2018 03:50:01 PM,41.807440,-87.703956,"(41.8074405, -87.703955849)"
1,3406613,HK456306,06/26/2004 12:40:00 PM,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,FALSE,FALSE,...,27,23,6,1152206,1906127,2004,02/28/2018 03:56:25 PM,41.898280,-87.716406,"(41.898279962, -87.716405505)"
2,8002131,HT233595,04/04/2011 05:45:00 AM,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,FALSE,FALSE,...,3,38,6,1177436,1876313,2011,02/10/2018 03:50:01 PM,41.815933,-87.624642,"(41.815933131, -87.624642127)"
3,7903289,HT133522,12/30/2010 04:30:00 PM,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,FALSE,FALSE,...,7,46,6,1194622,1850125,2010,02/10/2018 03:50:01 PM,41.743665,-87.562463,"(41.743665322, -87.562462756)"
4,10402076,HZ138551,02/02/2016 07:30:00 PM,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,FALSE,FALSE,...,15,66,6,1155240,1860661,2016,02/10/2018 03:50:01 PM,41.773455,-87.706480,"(41.773455295, -87.706480471)"
5,7732712,HS540106,09/29/2010 07:59:00 AM,006XX W CHICAGO AVE,810,THEFT,OVER $500,PARKING LOT/GARAGE(NON.RESID.),FALSE,FALSE,...,27,24,6,1171668,1905607,2010,02/10/2018 03:50:01 PM,41.896447,-87.644939,"(41.896446772, -87.644938678)"
6,10769475,HZ534771,11/30/2016 01:15:00 AM,050XX N KEDZIE AVE,810,THEFT,OVER $500,STREET,FALSE,FALSE,...,33,14,6,1154133,1933314,2016,02/10/2018 03:50:01 PM,41.972845,-87.708600,"(41.972844913, -87.708600079)"
7,4494340,HL793243,12/16/2005 04:45:00 PM,005XX E PERSHING RD,860,THEFT,RETAIL THEFT,GROCERY FOOD STORE,TRUE,FALSE,...,3,38,6,1180448,1879234,2005,02/28/2018 03:56:25 PM,41.823880,-87.613504,"(41.823879885, -87.613503857)"
8,3778925,HL149610,01/28/2005 05:00:00 PM,100XX S WASHTENAW AVE,810,THEFT,OVER $500,STREET,FALSE,FALSE,...,19,72,6,1160129,1838040,2005,02/28/2018 03:56:25 PM,41.711281,-87.689179,"(41.711280513, -87.689179097)"
9,3324217,HK361551,05/13/2004 02:15:00 PM,033XX W BELMONT AVE,820,THEFT,$500 AND UNDER,SMALL RETAIL STORE,FALSE,FALSE,...,35,21,6,1153590,1921084,2004,02/28/2018 03:56:25 PM,41.939296,-87.710923,"(41.939295821, -87.710923442)"


### Q3 How many crimes involve an arrest.

In [12]:
#Q3 
%sql select count(case_number) from CHICAGO_CRIME_DATA WHERE arrest = True

 * ibm_db_sa://gsv42017:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


1
163


In [45]:
sqlQ3 = "select count(case_number) from CHICAGO_CRIME_DATA WHERE arrest = True"

In [46]:
df_Q3 = pd.read_sql_query(sqlQ3,pconn)

In [47]:

df_Q3

,1
0,163.0


Q3 - Cross-checking in pandas

In [80]:
df_crime[df_crime['ARREST'] == "TRUE"].count()
df_crime['ARREST'].value_counts()

FALSE    370
TRUE     163
Name: ARREST, dtype: int64

### Q4. Which unique types of crimes (e.g. THEFT) have been recorded at a GAS STATION locations?

In [14]:
# Q4
%sql select distinct primary_type from CHICAGO_CRIME_DATA \
where lower(location_description) like 'gas%';

 * ibm_db_sa://gsv42017:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


primary_type
CRIMINAL TRESPASS
NARCOTICS
ROBBERY
THEFT


In [163]:
sqlQ4 = "select distinct primary_type from CHICAGO_CRIME_DATA where lower(location_description) like 'gas%'"

In [164]:
df_Q4 = pd.read_sql_query(sqlQ4,pconn)

In [165]:

df_Q4

,PRIMARY_TYPE
0,CRIMINAL TRESPASS
1,NARCOTICS
2,ROBBERY
3,THEFT


In [142]:
# verifying from pandas -- Q4


In [120]:
import re
pattern = re.compile("gas",flags=re.IGNORECASE)

In [155]:
df_crime_type = df_crime[df_crime['LOCATION_DESCRIPTION'].str.contains(pattern) == True]

In [156]:
df_crime_type

,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,...,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,UPDATEDON,LATITUDE,LONGITUDE,LOCATION
69,7505841,HS308747,05/15/2010 09:15:00 AM,004XX E 95TH ST,890,THEFT,FROM BUILDING,GAS STATION,FALSE,FALSE,...,9.0,49.0,6,1180667.0,1842071.0,2010,02/10/2018 03:50:01 PM,41.721896,-87.613841,"(41.721895856, -87.613841024)"
71,4153041,HL005270,07/12/2005 11:00:00 PM,008XX N CICERO AVE,820,THEFT,$500 AND UNDER,GAS STATION,FALSE,FALSE,...,37.0,25.0,6,1144207.0,1904963.0,2005,02/10/2018 03:50:01 PM,41.895240,-87.745815,"(41.895239896, -87.745814792)"
288,5844831,HN647052,10/14/2007 04:31:26 AM,062XX S DREXEL AVE,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,GAS STATION,TRUE,FALSE,...,20.0,42.0,18,1183081.0,1863726.0,2007,02/28/2018 03:56:25 PM,41.781264,-87.604327,"(41.781263748, -87.60432695)"
459,3046354,HJ753917,11/12/2003 01:40:00 AM,052XX W ADDISON ST,031A,ROBBERY,ARMED: HANDGUN,GAS STATION,FALSE,FALSE,...,38.0,15.0,3,1140892.0,1923436.0,2003,02/28/2018 03:56:25 PM,41.945993,-87.757535,"(41.945993442, -87.757534561)"
463,8875058,HV548880,11/05/2012 11:15:00 AM,067XX S STATE ST,330,ROBBERY,AGGRAVATED,GAS STATION,FALSE,FALSE,...,6.0,69.0,3,1177438.0,1860407.0,2012,02/10/2018 03:50:01 PM,41.772285,-87.625115,"(41.772285481, -87.625115462)"
474,5827212,HN637742,10/09/2007 12:20:00 PM,025XX N PULASKI RD,1330,CRIMINAL TRESPASS,TO LAND,GAS STATION,TRUE,FALSE,...,31.0,22.0,26,1149288.0,1916658.0,2007,02/28/2018 03:56:25 PM,41.927235,-87.726850,"(41.927235118, -87.726849666)"


In [131]:
df_crime_type['PRIMARY_TYPE'].nunique()

4

### Q5. In the CENUS_DATA table list all community areas whose names start with the letter ‘B’.

In [15]:
#Q5

%sql select COMMUNITY_AREA_NAME from CENSUS_DATA where upper(COMMUNITY_AREA_NAME) like 'B%';

 * ibm_db_sa://gsv42017:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name
Belmont Cragin
Burnside
Brighton Park
Bridgeport
Beverly


In [167]:
sqlQ5 = "select COMMUNITY_AREA_NAME from CENSUS_DATA where upper(COMMUNITY_AREA_NAME) like 'B%'"

In [168]:
df_Q5 = pd.read_sql_query(sqlQ5,pconn)

In [169]:
df_Q5

,COMMUNITY_AREA_NAME
0,Belmont Cragin
1,Burnside
2,Brighton Park
3,Bridgeport
4,Beverly


In [170]:
df_census[df_census['COMMUNITY_AREA_NAME'].str.contains(r'^B.*') == True]

,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED,PERCENT_HOUSEHOLDS_BELOW_POVERTY,PERCENT_AGED_16__UNEMPLOYED,PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,PERCENT_AGED_UNDER_18_OR_OVER_64,PER_CAPITA_INCOME,HARDSHIP_INDEX
18,19.0,Belmont Cragin,10.8,18.7,14.6,37.3,37.3,15461,70.0
46,47.0,Burnside,6.8,33.0,18.6,19.3,42.7,12515,79.0
57,58.0,Brighton Park,14.4,23.6,13.9,45.1,39.3,13089,84.0
59,60.0,Bridgeport,4.5,18.9,13.7,22.2,31.3,22694,43.0
71,72.0,Beverly,0.9,5.1,8.0,3.7,40.5,39523,12.0


### Q6. List the schools in community areas 10 to 15 that are healthy school certified.

In [17]:
#Q6 
%sql select Name_of_School, Community_Area_Number from CHICAGO_PUBLIC_SCHOOLS \
    where lower(healthy_school_certified) = 'yes' \
         and community_area_number between 10 and 15;

 * ibm_db_sa://gsv42017:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school,community_area_number
Rufus M Hitch Elementary School,10


In [212]:
sqlQ6 = "select Name_of_School, Community_Area_Number from CHICAGO_PUBLIC_SCHOOLS where lower(healthy_school_certified) = 'yes' \
         and community_area_number between 10 and 15"

In [213]:
df_Q6 = pd.read_sql_query(sqlQ6,pconn)

In [214]:
df_Q6

,NAME_OF_SCHOOL,COMMUNITY_AREA_NUMBER
0,Rufus M Hitch Elementary School,10


In [ ]:
# cross checking in Pandas

In [232]:
df_q6_school = df_school[df_school.HEALTHY_SCHOOL_CERTIFIED.str.contains(r'(^[Yy.*])') == True]
         

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


In [234]:
df_q6_school[df_q6_school.COMMUNITY_AREA_NUMBER.between(10,15)]

,School_ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,Collaborative_Name,Adequate_Yearly_Progress_Made_,Track_Schedule,CPS_Performance_Policy_Status,CPS_Performance_Policy_Level,HEALTHY_SCHOOL_CERTIFIED,Safety_Icon,SAFETY_SCORE,Family_Involvement_Icon,Family_Involvement_Score,Environment_Icon,Environment_Score,Instruction_Icon,Instruction_Score,Leaders_Icon,Leaders_Score,Teachers_Icon,Teachers_Score,Parent_Engagement_Icon,Parent_Engagement_Score,Parent_Environment_Icon,Parent_Environment_Score,AVERAGE_STUDENT_ATTENDANCE,Rate_of_Misconducts__per_100_students_,Average_Teacher_Attendance,Individualized_Education_Program_Compliance_Rate,Pk_2_Literacy__,Pk_2_Math__,Gr3_5_Grade_Level_Math__,Gr3_5_Grade_Level_Read__,Gr3_5_Keep_Pace_Read__,Gr3_5_Keep_Pace_Math__,Gr6_8_Grade_Level_Math__,Gr6_8_Grade_Level_Read__,Gr6_8_Keep_Pace_Math_,Gr6_8_Keep_Pace_Read__,Gr_8_Explore_Math__,Gr_8_Explore_Read__,ISAT_Exceeding_Math__,ISAT_Exceeding_Reading__,ISAT_Value_Add_Math,ISAT_Value_Add_Read,ISAT_Value_Add_Color_Math,ISAT_Value_Add_Color_Read,Students_Taking__Algebra__,Students_Passing__Algebra__,9th Grade EXPLORE (2009),9th Grade EXPLORE (2010),10th Grade PLAN (2009),10th Grade PLAN (2010),Net_Change_EXPLORE_and_PLAN,11th Grade Average ACT (2011),Net_Change_PLAN_and_ACT,College_Eligibility__,Graduation_Rate__,College_Enrollment_Rate__,COLLEGE_ENROLLMENT,General_Services_Route,Freshman_on_Track_Rate__,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,Ward,Police_District,Location
471,609995,Rufus M Hitch Elementary School,ES,5625 N McVicker Ave,Chicago,IL,60646,(773) 534-1189,http://schoolreports.cps.edu/SchoolProgressRep...,O'Hare Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 1,Yes,Strong,64.0,Strong,77,Average,55.0,Average,58.0,Strong,65,Strong,70,Strong,55,Strong,54,95.10%,3.7,96.60%,98.60%,73.7,27.6,62.2,51.7,50,67.8,56.5,54.8,52,63.3,22.9,37.5,29.5,24.7,-0.1,0.0,Yellow,Yellow,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,542,30,NDA,1134940.195,1937129.571,41.983678,-87.779086,10,NORWOOD PARK,45,16,"(41.98367756, -87.77908614)"


### Q7 What is the average school Safety Score?

In [23]:
#Q7
%sql Select median(safety_score) from chicago_public_schools;
%sql Select avg(safety_score) from chicago_public_schools

 * ibm_db_sa://gsv42017:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.
 * ibm_db_sa://gsv42017:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


1
49.504873


In [239]:
sqlQ7 = "Select avg(safety_score) from chicago_public_schools"

In [240]:
df_Q7 = pd.read_sql_query(sqlQ7,pconn)


In [241]:
df_Q7

,1
0,49.504873


In [246]:
df_school.SAFETY_SCORE.mean()

49.50487329434698

### Q8 Find the top 5 Community Areas by average College Enrollment [number of students].

In [36]:
#Q8
%sql select community_area_name, community_area_number, college_enrollment\
        from chicago_public_schools order by college_enrollment desc limit 5;


## Question is confusing: Correct query below:
%sql select community_area_name, avg(college_enrollment) as "Avg_Enrollment" \
        from chicago_public_schools \
        group by community_area_name \
        order by "Avg_Enrollment" desc limit 5;

 * ibm_db_sa://gsv42017:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.
 * ibm_db_sa://gsv42017:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name,Avg_Enrollment
ARCHER HEIGHTS,2411.500000
MONTCLARE,1317.000000
WEST ELSDON,1233.333333
BRIGHTON PARK,1205.875000
BELMONT CRAGIN,1198.833333


In [267]:
sqlQ8 = "select community_area_name, community_area_number, college_enrollment\
        from chicago_public_schools order by college_enrollment desc limit 5"

In [270]:
df_Q8 = pd.read_sql(sqlQ8,pconn)

In [271]:
df_Q8

,COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER,COLLEGE_ENROLLMENT
0,NORTH CENTER,5,4368
1,ARCHER HEIGHTS,57,3320
2,NORWOOD PARK,10,2922
3,BRIGHTON PARK,58,2883
4,IRVING PARK,16,2366


In [277]:
# cross-checking in pandas

#df_school['COMMUNITY_AREA_NAME'].
pdf_Q8 = df_school.sort_values(by='COLLEGE_ENROLLMENT', ascending=False).head(5)

In [280]:
pdf_Q8[['NAME_OF_SCHOOL','COMMUNITY_AREA_NAME','COLLEGE_ENROLLMENT']]

,NAME_OF_SCHOOL,COMMUNITY_AREA_NAME,COLLEGE_ENROLLMENT
6,Albert G Lane Technical High School,NORTH CENTER,4368
357,Marie Sklodowska Curie Metropolitan High School,ARCHER HEIGHTS,3320
545,William Howard Taft High School,NORWOOD PARK,2922
508,Thomas Kelly High School,BRIGHTON PARK,2883
60,Carl Schurz High School,IRVING PARK,2366


### Q9. Use a sub-query todeterminewhich Community Area has the least value for school Safety Score?

In [25]:
#Q9
%sql select name_of_school, community_area_name, safety_score from chicago_public_schools \
   where safety_score = (select min(safety_score) from chicago_public_schools)

 * ibm_db_sa://gsv42017:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school,community_area_name,safety_score
Edmond Burke Elementary School,WASHINGTON PARK,1


In [389]:
sqlQ9 = "select name_of_school, community_area_name, safety_score from chicago_public_schools \
   where safety_score = (select min(safety_score) from chicago_public_schools)"


In [313]:
    
## THIS IS INCORRECT
#sqlQ9 = "select distinct community_area_name from chicago_public_schools \
#   where \
#   exists ( select safety_score from chicago_public_schools where safety_score = 1);"

In [390]:
df_Q9 = pd.read_sql(sqlQ9, pconn)

In [391]:
df_Q9

,NAME_OF_SCHOOL,COMMUNITY_AREA_NAME,SAFETY_SCORE
0,Edmond Burke Elementary School,WASHINGTON PARK,1


In [388]:
# cross-checking in pandas
#df_school.sort_values(by='SAFETY_SCORE',ascending=True).head(5)
#df_school['SAFETY_SCORE'].min()
df_school[df_school.SAFETY_SCORE == 1]



77

### Q10. [Without using an explicit JOIN operator] Find the Per Capita Income of the Community Area which has a school Safety Score of 1.

In [26]:
#Q 10
%sql select C.per_capita_income, S.safety_score, S.community_area_name \
    from census_data as C, chicago_public_schools as S \
    where C.community_area_number = S.community_area_number and S.safety_score = 1;

 * ibm_db_sa://gsv42017:***@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
Done.


per_capita_income,safety_score,community_area_name
13785,1,WASHINGTON PARK


In [392]:
sqlQ10 = "select C.per_capita_income, S.safety_score, S.community_area_name \
    from census_data as C, chicago_public_schools as S \
    where C.community_area_number = S.community_area_number and S.safety_score = 1"

In [393]:
df_Q10 = pd.read_sql(sqlQ10,pconn)

In [444]:
df_Q10

,PER_CAPITA_INCOME,SAFETY_SCORE,COMMUNITY_AREA_NAME
0,13785,1,WASHINGTON PARK


In [443]:
# Q10. Cross checking in pandas
result = df_census.merge(df_school,on=['COMMUNITY_AREA_NUMBER'], how='inner')
result[result.SAFETY_SCORE == 1].PER_CAPITA_INCOME

321    13785
Name: PER_CAPITA_INCOME, dtype: int64